# Apriori算法

![Apriori算法流程](Apriori算法流程.png)

In [17]:
def loadDataSet():
    return [[1,3,5],[2,3,5],[1,2,3,5],[2,5]]


def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset,C1))


# 扫描
def scanD(dataSet,CK,minSupport):
    ssCnt = {}
    for tid in dataSet:
        for can in CK:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1 
    returnList = []
    supportData = {}    
    numItems = float(len(list(dataSet)))
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            returnList.insert(0,key)
        supportData[key] = support
    return returnList,supportData


# 拼接
def aprioriGen(Lk,k):
    returnList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            if L1 == L2:
                returnList.append( Lk[i] | Lk[j])
    return returnList


# 主算法
def apriori(dataSet,minSupport=0.5):
    C1 = createC1(dataSet)
    L1,supportData = scanD(dataSet,C1,minSupport)
    L = [L1]
    k = 2
    while(len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2],k)
        Lk,supk = scanD(dataSet,Ck,minSupport)
        supportData.update(supk)
        L.append(Lk)
        k += 1
    return L,supportData


# 构建整体规则
def generateRules(L,supportData,minConf=0.6):
    returnList = []
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            rulessFromConseq(freqSet,H1,supportData,returnList,minConf)

            
def rulessFromConseq(freqSet,H,supportData,returnList,minConf=0.6):
    m = len(H[0])
    while (len(freqSet)>m):
        H = calConf(freqSet,H,supportData,returnList,minConf)
        if (len(H)>1):
            aprioriGen(H,m+1)
            m += 1
        else:
            break 


def calConf(freqSet,H,supportData,returnList,minConf=0.6):
    prunedh = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq,'-->',conseq,'conf',conf)
            returnList.append((freqSet - conseq,conseq,conf))
            prunedh.append(conseq)
    return  prunedh 


if __name__ == '__main__':
    dataSet = loadDataSet()
    L,support = apriori(dataSet)
    i = 0
    for freq in L:
        print("项数",i+1,'：',freq)
        i += 1
    rules = generateRules(L,support,minConf=0.5)
    

项数 1 ： [frozenset({2}), frozenset({5}), frozenset({3}), frozenset({1})]
项数 2 ： [frozenset({2, 3}), frozenset({2, 5}), frozenset({1, 3}), frozenset({1, 5}), frozenset({3, 5})]
项数 3 ： [frozenset({2, 3, 5}), frozenset({1, 3, 5})]
项数 4 ： []
frozenset({3}) --> frozenset({2}) conf 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf 0.75
frozenset({2}) --> frozenset({5}) conf 1.0
frozenset({3}) --> frozenset({1}) conf 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf 1.0
frozenset({5}) --> frozenset({1}) conf 0.5
frozenset({1}) --> frozenset({5}) conf 1.0
frozenset({5}) --> frozenset({3}) conf 0.75
frozenset({3}) --> frozenset({5}) conf 1.0
frozenset({3, 5}) --> frozenset({2}) conf 0.6666666666666666
frozenset({2, 5}) --> frozenset({3}) conf 0.6666666666666666
frozenset({2, 3}) --> frozenset({5}) conf 1.0
frozenset({3, 5}) --> frozenset({2}) conf 0.6666666666666666
frozenset({2, 5}) --> frozenset({3}) conf 0.6666666666666